<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#modeling" data-toc-modified-id="modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>modeling</a></span></li></ul></div>

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src/"))

In [ ]:
from load_data import LoadData
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from nklearn.lgbm_util.modeling import CvEnsembleModeling
import numpy as np
import pandas as pd
import uuid

In [ ]:
uid = uuid.uuid1()

In [ ]:
alg = "stack"

In [ ]:
docs = "../docs/"
finfo = docs + "info/" + "ddl.csv"

data = "../data/"
data_in, data_mid, data_out = [data + fld for fld in ["in/", "mid/", "out/"]]

fin, ftest = data_in + "train_data.csv", data_in + "test_data.csv"
fmid = data_mid + "%s_%s.csv"%(alg, uid)
fout = data_out + "%s_%s.csv"%(alg, uid)

models = "../models/"
mout = models + "%s_%s.pkl"%(alg, uid)

loader = LoadData(fin, ftest, finfo)

In [ ]:
X, y, test_X = loader.get_features_v1()

In [ ]:
y = y.astype(float)

In [ ]:
scores = {}
for p in os.listdir(data_mid)[1:]:
    scores[p] = np.loadtxt(data_mid+p)
df_score = pd.DataFrame(scores)

In [ ]:
from sklearn.externals import joblib

In [ ]:
scores = {}
for p in os.listdir(data_mid)[1:]:
    m = joblib.load("../models/" + p.split(".")[-2] + ".pkl")
#     print(p.split(".")[-2].split("_")[0])
    if p.split(".")[-2].split("_")[0] != "svc":
        scores[p] = m.predict_proba(test_X)[:,1]
    else:
        scores[p] = m.predict(test_X)
        
df_score_test = pd.DataFrame(scores)

In [ ]:
X = pd.concat([X, df_score],1)

In [ ]:
test_X = pd.concat([test_X, df_score_test],1)

# modeling

In [ ]:
opt = CvEnsembleModeling()

In [ ]:
models, best_scores = opt.opt_fit(X=X, y=y, n_trials=20)

In [ ]:
w = 1 - np.array(best_scores)

In [ ]:
def predict(X, models, weight):
    preds = []
    X = np.array(X)
    for model, w in zip(models, weight):
        preds.append(model.predict(X)*w)
    preds = np.array(preds)
    return preds.sum(0)/sum(weight)


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
p = predict(X, models, w)

In [ ]:
roc_auc_score(y_true=y, y_score=p)

In [ ]:
best_thre, max_acc = 0,0
for thre in range(100):
    b = p >= thre/100
    b = np.vectorize(int)(b)
    acc = sum(b == y)/len(y)
    if acc > max_acc:
        best_thre = thre/100
        max_acc = acc
    print(thre, acc)

In [ ]:
p = predict(test_X, models, w)

In [ ]:
b = p >= best_thre
Y = np.vectorize(int)(b)

In [ ]:
test_X["Y"] = Y

In [ ]:
test_X["ID"] = test_X.index

In [ ]:
test_X[["ID", "Y"]].to_csv(fout, index=False)

In [ ]:
joblib.dump(models, mout)

In [ ]:
X[y==1][fmid].hist()

In [ ]:
X[y==0][fmid].hist()